In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import random
from torch.optim import AdamW
import torch.nn as nn

from transformers import *
from tokenizers import *

from bertPhoneme import BertEmbeddingsV2, BertModelV2, BertForMaskedLMV2, BertConfigV2, MaskedLMWithProsodyOutput

2025-06-03 03:34:47.882906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748921687.896996 2398124 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748921687.901316 2398124 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748921687.913539 2398124 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748921687.913552 2398124 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748921687.913554 2398124 computation_placer.cc:177] computation placer alr

[2025-06-03 03:34:51,072] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/anaconda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/anaconda/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: /usr/local/cuda-12.4/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/opt/anaconda/compiler_compat/ld: /usr/local/cuda-12.4/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@C

In [3]:
import json
import random
from datasets import load_dataset

In [12]:
file_path = "/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1.jsonl"
phonemes = set()

with open(file_path, 'r') as f:
    for i, line in enumerate(f):
        if i >= 5000:
            break
        phonemes.update(set(json.loads(line)['phoneme']))
    print(len(phonemes))

70


In [13]:
# phonemes.update(['UNK', 'SIL'])
phonemes.update(['UNK'])

In [14]:
phoneme_vocab = {p: i for i, p in enumerate(phonemes)}

In [15]:
print(phoneme_vocab)

{'AH2': 0, 'ER1': 1, 'IY0': 2, 'AA1': 3, 'AW1': 4, 'IH1': 5, 'UH1': 6, 'EY1': 7, 'IH2': 8, 'OY1': 9, 'AH0': 10, 'EH1': 11, 'P': 12, 'UNK': 13, 'T': 14, 'AA0': 15, 'AO0': 16, 'EH2': 17, 'N': 18, 'IY1': 19, 'UW1': 20, 'TH': 21, 'AH1': 22, 'IY2': 23, 'DH': 24, 'AY1': 25, 'M': 26, 'Y': 27, 'OW0': 28, 'L': 29, 'R': 30, 'UW2': 31, 'EY0': 32, 'D': 33, 'EH0': 34, 'NG': 35, 'AW2': 36, 'AE0': 37, 'Z': 38, 'V': 39, 'AO1': 40, 'AY2': 41, 'IH0': 42, 'ER0': 43, 'UH2': 44, 'S': 45, 'ZH': 46, 'W': 47, 'spn': 48, 'G': 49, 'OW1': 50, 'OY2': 51, 'K': 52, 'B': 53, 'OW2': 54, 'CH': 55, 'AW0': 56, 'UW0': 57, 'HH': 58, 'AO2': 59, 'OY0': 60, 'AA2': 61, 'AY0': 62, 'SH': 63, 'AE1': 64, 'F': 65, 'ER2': 66, 'AE2': 67, 'EY2': 68, 'JH': 69, 'UH0': 70}


In [ ]:
phoneme_vocab_size = len(phoneme_vocab)
mask_token_id = phoneme_vocab["SIL"]
pad_token_id = 72
pad_cluster_id = 200

class HuggingFacePhonemeDataset(Dataset):
    def __init__(self, hf_dataset, vocab, mask_prob=0.15, max_length=512):
        self.dataset = hf_dataset
        self.vocab = vocab
        self.mask_prob = mask_prob
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]

        phonemes = sample["phoneme"]
        prosody_ids = sample["prosody_id_200"]

        input_ids = [self.vocab.get(p, self.vocab["UNK"]) for p in phonemes][:self.max_length]
        prosody_ids = prosody_ids[:self.max_length]

        labels = input_ids.copy()
        for i in range(len(input_ids)):
            if random.random() < self.mask_prob:
                labels[i] = input_ids[i]
                input_ids[i] = self.vocab["SIL"]

        pad_length = self.max_length - len(input_ids)
        input_ids.extend([pad_token_id] * pad_length)
        labels.extend([-100] * pad_length)
        prosody_ids.extend([pad_cluster_id] * pad_length)
        attention_mask = [1] * (self.max_length - pad_length) + [0] * pad_length
        
        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long),
            "prosody_ids": torch.tensor(prosody_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
        }

# from torch.utils.data import Dataset
# import torch
# import random

# class ChunkedPhonemeDataset(Dataset):
#     def __init__(self, hf_dataset, vocab, mask_prob=0.15, max_length=512):
#         self.vocab = vocab
#         self.mask_prob = mask_prob
#         self.max_length = max_length
#         self.pad_token_id = 72
#         self.pad_cluster_id = 200
#         self.mask_token_id = vocab["SIL"]
#         self.unk_token_id = vocab["UNK"]

#         # Preprocess into chunks
#         self.samples = []
#         for sample in hf_dataset:
#             phonemes = [vocab.get(p, self.unk_token_id) for p in sample["phoneme"]]
#             prosody_ids = sample["prosody_id_200"]

#             assert len(phonemes) == len(prosody_ids), "Mismatched phoneme and prosody lengths"

#             for i in range(0, len(phonemes), max_length):
#                 chunk_input_ids = phonemes[i:i+max_length]
#                 chunk_prosody = prosody_ids[i:i+max_length]

#                 self.samples.append((chunk_input_ids, chunk_prosody))

#     def __len__(self):
#         return len(self.samples)

#     def __getitem__(self, idx):
#         input_ids, prosody_ids = self.samples[idx]

#         # Apply MLM masking
#         labels = input_ids.copy()
#         for i in range(len(input_ids)):
#             if random.random() < self.mask_prob:
#                 labels[i] = input_ids[i]
#                 input_ids[i] = self.mask_token_id
#             else:
#                 labels[i] = -100

#         # Pad to max_length
#         pad_len = self.max_length - len(input_ids)
#         input_ids += [self.pad_token_id] * pad_len
#         labels += [-100] * pad_len
#         prosody_ids += [self.pad_cluster_id] * pad_len
#         attention_mask = [1] * (self.max_length - pad_len) + [0] * pad_len

#         return {
#             "input_ids": torch.tensor(input_ids, dtype=torch.long),
#             "labels": torch.tensor(labels, dtype=torch.long),
#             "prosody_ids": torch.tensor(prosody_ids, dtype=torch.long),
#             "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
#         }

In [16]:
phoneme_vocab_size = len(phoneme_vocab)
# mask_token_id = phoneme_vocab["SIL"]
pad_token_id = 71
pad_cluster_id = 201
mask_prosody_id = 200

class HuggingFacePhonemeDataset(Dataset):
    def __init__(self, hf_dataset, vocab, mask_prob=0.15, max_length=512):
        self.dataset = hf_dataset
        self.vocab = vocab
        self.mask_prob = mask_prob
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]

        phonemes = sample["phoneme"]
        prosody_ids = sample["prosody_id_200"]

        # Tokenize and truncate
        input_ids = [self.vocab.get(p, self.vocab["UNK"]) for p in phonemes][:self.max_length]
        prosody_ids = prosody_ids[:self.max_length]

        # Create prosody_labels and apply masking
        prosody_labels = prosody_ids.copy()
        for i in range(len(prosody_ids)):
            if random.random() < self.mask_prob:
                prosody_labels[i] = prosody_ids[i]       # Keep true label for loss
                prosody_ids[i] = mask_prosody_id         # Mask token
            else:
                prosody_labels[i] = -100                 # Ignore in loss

        # Padding
        pad_length = self.max_length - len(input_ids)
        input_ids.extend([pad_token_id] * pad_length)
        prosody_ids.extend([pad_cluster_id] * pad_length)
        prosody_labels.extend([-100] * pad_length)
        attention_mask = [1] * (self.max_length - pad_length) + [0] * pad_length

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "prosody_ids": torch.tensor(prosody_ids, dtype=torch.long),
            "prosody_labels": torch.tensor(prosody_labels, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
        }

In [8]:
hf_dataset = load_dataset("json", data_files="/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1_1subset.jsonl", split="train")
hf_dataset = hf_dataset.select(range(4900))


In [9]:
len(hf_dataset)

4900

In [10]:
hf_dataset

Dataset({
    features: ['name', 'phoneme', 'prosody_id_50', 'prosody_id_100', 'prosody_id_200', 'prosody_id_500', 'prosody_id_1000'],
    num_rows: 4900
})

In [19]:
train_dataset = HuggingFacePhonemeDataset(hf_dataset, phoneme_vocab)

In [20]:
train_dataset[1]

{'input_ids': tensor([58, 10, 29, 50, 10, 18, 33, 47, 11, 29, 52, 10, 26, 14, 42, 24,  2, 50,
         18, 29,  2, 30, 19, 29, 52, 27,  6, 30, 14, 10, 53, 40, 30, 33, 43, 29,
         41, 18, 12, 66, 45, 10, 18, 64, 29, 42, 14,  2, 33, 42, 45, 40, 30, 33,
         43, 48, 25, 26, 48, 10, 33, 11, 29, 30, 28, 38,  3, 30,  2, 28, 18,  4,
         25, 26, 48, 64, 18, 33, 25, 39, 10, 18, 18, 50, 42, 35, 29,  2, 29,  5,
         39, 33, 47, 42, 21, 53, 40, 30, 33, 43, 29, 41, 18, 12, 66, 45, 10, 18,
         64, 29, 42, 14,  2, 33, 42, 45, 40, 30, 33, 43, 65, 43, 26, 25, 42, 18,
         14, 25, 43, 29, 25, 65,  5, 14, 45, 42, 49, 38, 40, 45, 14, 42, 35,  5,
         14, 45, 10, 52,  3, 18, 45, 14, 10, 18, 14, 42, 26, 50, 63, 10, 18, 10,
         29, 30, 50, 29, 43, 52, 50, 45, 14, 43, 10, 65, 11, 52, 14, 42, 35, 11,
         39, 30,  2, 12,  3, 30, 14, 10, 39, 29, 25, 65, 42, 18, 52, 29, 20, 33,
         42, 35, 30,  2, 29,  7, 63, 10, 18, 63,  8, 12, 45, 55,  5, 29, 33, 30,
         10, 18

In [21]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [22]:
vocab_size = len(phoneme_vocab.keys())
phoneme_vocab_size = len(phoneme_vocab)
max_length = 512


print(vocab_size)

model_config = BertConfigV2(
    vocab_size=vocab_size+1,
    pad_token_id=pad_token_id,
    pad_cluster_id=200,
    hidden_size=128,
    num_hidden_layers=2,
    num_attention_heads=2,
    intermediate_size=512,
    max_position_embeddings=max_length,
    prosody_cluster_size=200 + 2
)

model = BertForMaskedLMV2(config=model_config)

# BERT-Base	768	12	12	3072
# BERT-Small 512	4	8	2048
# BERT-Mini	256	4	4	1024
# BERT-Tiny	128	2	2	512

71


In [23]:
model

BertForMaskedLMV2(
  (bert): BertModelV2(
    (embeddings): BertEmbeddingsV2(
      (word_embeddings): Embedding(72, 128, padding_idx=71)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (prosody_embeddings): Embedding(202, 128, padding_idx=200)
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)

mlm_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
prosody_loss_fn = nn.CrossEntropyLoss()

num_epochs = 3
model.train()

for epoch in range(num_epochs):
    total_loss = 0

    print(f"\n Epoch {epoch + 1} starting...\n")

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)
        prosody_ids = batch["prosody_ids"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, prosody_ids=prosody_ids)

        mlm_loss = mlm_loss_fn(outputs.logits.view(-1, phoneme_vocab_size + 1), labels.view(-1))
        prosody_loss = prosody_loss_fn(outputs.prosody_logits.view(-1, model.config.prosody_cluster_size), prosody_ids.view(-1))

        total_batch_loss = mlm_loss + prosody_loss
        total_batch_loss.backward()
        optimizer.step()

        total_loss += total_batch_loss.item()

        # Report every 100 steps
        if (step + 1) % 100 == 0:
            avg_loss = total_loss / (step + 1)
            print(f"  Step {step + 1:>4} | Avg Loss: {avg_loss:.4f}")

    avg_epoch_loss = total_loss / len(train_loader)
    print(f"\n Epoch {epoch + 1} completed. Avg Loss: {avg_epoch_loss:.4f}")


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)

mlm_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
prosody_loss_fn = nn.CrossEntropyLoss()

num_epochs = 3
model.train()

for epoch in range(num_epochs):
    total_loss = 0

    print(f"\n Epoch {epoch + 1} starting...\n")

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        labels = batch["prosody_labels"].to(device)
        prosody_ids = batch["prosody_ids"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, prosody_ids=prosody_ids)

        mlm_loss = mlm_loss_fn(outputs.logits.view(-1, phoneme_vocab_size + 1), labels.view(-1))
        prosody_loss = prosody_loss_fn(outputs.prosody_logits.view(-1, model.config.prosody_cluster_size), prosody_ids.view(-1))

        total_batch_loss = mlm_loss + prosody_loss
        total_batch_loss.backward()
        optimizer.step()

        total_loss += total_batch_loss.item()

        # Report every 100 steps
        if (step + 1) % 100 == 0:
            avg_loss = total_loss / (step + 1)
            print(f"  Step {step + 1:>4} | Avg Loss: {avg_loss:.4f}")

    avg_epoch_loss = total_loss / len(train_loader)
    print(f"\n Epoch {epoch + 1} completed. Avg Loss: {avg_epoch_loss:.4f}")



 Epoch 1 starting...



/opt/anaconda/lib/python3.11/site-packages/transformers/modeling_utils.py:1575: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [21,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [28,0,0] Assertion `t >= 0 && t < n_classes` failed.


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [109]:
model.resize_token_embeddings(len(tokenizer))

NameError: name 'tokenizer' is not defined

In [16]:
# Load full dataset first
hf_dataset_full = load_dataset(
    "json",
    data_files="/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1.jsonl",
    split="train"
)

# Get next 100 for testing
hf_test_dataset = hf_dataset_full.select(range(49000, 50000))


Loading dataset shards:   0%|          | 0/60 [00:00<?, ?it/s]

In [17]:
def evaluate_masked_sample(
    phonemes, prosody_ids, model, vocab,
    pad_token_id=72, pad_cluster_id=200, max_length=512, mask_token="SIL"
):
    model.eval()
    mask_token_id = vocab[mask_token]
    unk_token_id = vocab["UNK"]
    id2phoneme = {v: k for k, v in vocab.items()}

    # === 1. Convert and mask ===
    input_ids = [vocab.get(p, unk_token_id) for p in phonemes][:max_length]
    prosody_ids = prosody_ids[:max_length]
    labels = input_ids.copy()

    mask_positions = []
    for i in range(len(input_ids)):
        if random.random() < 0.15:  # Mask 20% of the tokens
            input_ids[i] = mask_token_id
            mask_positions.append(i)
        else:
            labels[i] = -100  # Ignore in loss

    # === 2. Padding ===
    pad_len = max_length - len(input_ids)
    input_ids.extend([pad_token_id] * pad_len)
    labels.extend([-100] * pad_len)
    prosody_ids.extend([pad_cluster_id] * pad_len)
    attention_mask = [1] * (max_length - pad_len) + [0] * pad_len

    # === 3. Tensorize and predict ===
    input_ids = torch.tensor([input_ids], dtype=torch.long).to(model.device)
    labels = torch.tensor([labels], dtype=torch.long).to(model.device)
    prosody_ids = torch.tensor([prosody_ids], dtype=torch.long).to(model.device)
    attention_mask = torch.tensor([attention_mask], dtype=torch.long).to(model.device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, prosody_ids=prosody_ids, attention_mask=attention_mask)

    pred_phonemes = torch.argmax(outputs.logits, dim=-1)[0]         # [seq_len]
    pred_prosody = torch.argmax(outputs.prosody_logits, dim=-1)[0]  # [seq_len]

    # === 4. Compare and collect ===
    results = []
    correct_phoneme = 0
    correct_prosody = 0

    for i in mask_positions:
        gt_ph_id = labels[0, i].item()
        pred_ph_id = pred_phonemes[i].item()

        gt_pr_id = prosody_ids[0, i].item()
        pred_pr_id = pred_prosody[i].item()

        phoneme_correct = gt_ph_id == pred_ph_id
        prosody_correct = gt_pr_id == pred_pr_id

        if phoneme_correct:
            correct_phoneme += 1
        if prosody_correct:
            correct_prosody += 1

        results.append({
            "position": i,
            "phoneme_correct": phoneme_correct,
            "phoneme_gt": id2phoneme.get(gt_ph_id, "UNK"),
            "phoneme_pred": id2phoneme.get(pred_ph_id, "UNK"),
            "phoneme_gt_id": gt_ph_id,
            "phoneme_pred_id": pred_ph_id,

            "prosody_correct": prosody_correct,
            "prosody_gt_id": gt_pr_id,
            "prosody_pred_id": pred_pr_id,
        })

    total = len(mask_positions)
    phoneme_accuracy = correct_phoneme / total if total > 0 else None
    prosody_accuracy = correct_prosody / total if total > 0 else None

    return results, phoneme_accuracy, prosody_accuracy


In [18]:
sample = hf_test_dataset[15]
phonemes = sample["phoneme"]
prosody = sample["prosody_id_200"]

results, phoneme_acc, prosody_acc = evaluate_masked_sample(phonemes, prosody, model, phoneme_vocab)

print(f"\nPhoneme Accuracy: {phoneme_acc:.2%}" if phoneme_acc is not None else "No masked phoneme positions.")
print(f"Prosody Accuracy: {prosody_acc:.2%}" if prosody_acc is not None else "No masked prosody positions.")

for r in results:
    print(f"Pos {r['position']:>3} | Phoneme: Pred={r['phoneme_pred']} GT={r['phoneme_gt']} → {'True' if r['phoneme_correct'] else 'False'}")
          #f" | Prosody: Pred={r['prosody_pred_id']} GT={r['prosody_gt_id']} → {'True' if r['prosody_correct'] else 'False'}")



Phoneme Accuracy: 36.84%
Prosody Accuracy: 100.00%
Pos  30 | Phoneme: Pred=EH1 GT=AO1 → False
Pos  33 | Phoneme: Pred=IH0 GT=IH0 → True
Pos  52 | Phoneme: Pred=T GT=D → False
Pos  60 | Phoneme: Pred=AH1 GT=R → False
Pos  61 | Phoneme: Pred=N GT=IH1 → False
Pos  62 | Phoneme: Pred=S GT=S → True
Pos  64 | Phoneme: Pred=AH0 GT=AH0 → True
Pos  65 | Phoneme: Pred=N GT=N → True
Pos  68 | Phoneme: Pred=AH1 GT=EH1 → False
Pos  71 | Phoneme: Pred=NG GT=K → False
Pos  74 | Phoneme: Pred=AH0 GT=AH0 → True
Pos  83 | Phoneme: Pred=ER0 GT=AH0 → False
Pos  93 | Phoneme: Pred=L GT=V → False
Pos  96 | Phoneme: Pred=EH1 GT=AY1 → False
Pos 109 | Phoneme: Pred=OY1 GT=ER1 → False
Pos 116 | Phoneme: Pred=Y GT=D → False
Pos 125 | Phoneme: Pred=N GT=R → False
Pos 132 | Phoneme: Pred=EH1 GT=AO1 → False
Pos 135 | Phoneme: Pred=IH0 GT=IH0 → True
Pos 153 | Phoneme: Pred=Y GT=Y → True
Pos 159 | Phoneme: Pred=EH1 GT=IH1 → False
Pos 161 | Phoneme: Pred=T GT=K → False
Pos 164 | Phoneme: Pred=D GT=D → True
Pos 170 | 